In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.getcwd(),
    indicator=".project-root",
    pythonpath=True,
    dotenv=True,
)

In [ ]:
%matplotlib inline

import src.utils
import src.predict
import omegaconf

# Configuration


In [ ]:
# to use ensemble: i) set multiple model_dirs, ii) set to parent directory of several model_dirs, iii) set to glob pattern (e.g. logs/train/multiruns/2023_10-14_*)
model_dir = ["logs/train/multiruns/2023-10-13_15-25-56/0", "logs/train/multiruns/2023-10-13_15-25-56/1"]
model_dir = [src.utils.get_absolute_project_path(md) for md in model_dir]

config_path = os.path.join("..", "..", "configs", "predict.yaml") # NB: relative to <project_root>/src/utils (must be relative path)

config_overrides_dot = [# same notation as for cli overrides (dot notation). Useful for changing whole modules, e.g. change which datamodule file is loaded
    "++predict.presenter=null",  # set presenter to null to return figures
    "++extras.disable_pytorch_lightning_output=True",
    "++datamodule.crop_data_range=null",
    "++datamodule.check_for_nan=False"
]
config_overrides_dict = dict(model_dir=model_dir)  # Dictionary with overrides. Useful for larger changes/additions/deletions that does not exist as entire files.

cfg = src.utils.initialize_hydra(config_path, config_overrides_dot, config_overrides_dict, return_hydra_config=True, print_config=False)  # print config to inspect if all settings are as expected

In [ ]:
object_dict = src.utils.initialize_saved_objects(cfg)
model, datamodule = object_dict["model"], object_dict["datamodule"]

In [ ]:
with omegaconf.open_dict(cfg):
    cfg.predict.kwargs.n = 24
    cfg.predict.indices = [0.5, 0.75]

# Predict
The src.predict.run function returns the configured metrics evaluated over the predictions, the predictions themselves, and any figures configured.

In [ ]:
metric_dict, predictions, figs = src.predict.run(cfg, datamodule, model)